## 通過 Google Cloud 進行驗證

In [6]:
import os

# 請確保 'client_secret.json' 檔案存在於目前目錄或指定路徑
client_secret_file = "client_secret.json"

if not os.path.exists(client_secret_file):
    raise FileNotFoundError(f"client_secret.json 文件不在路徑中：{client_secret_file}")

# 執行 gcloud 登入
os.system(
    f"gcloud auth application-default login --client-id-file={client_secret_file} --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'"
)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=826653416557-5f254qj34fu2kir6pl0bqin5jf8uifaf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.tuning&state=hdSb4WOhTDa7EAbO9OdjzHhOqlx8KF&access_type=offline&code_challenge=s0UNoa1B24hQ-U-QAc3X8htiTUzI7dPXvkOz7x4GHw4&code_challenge_method=S256


Credentials saved to file: [/Users/samhsiao/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).


0

## 透過自動化腳本進行驗證

單獨生成哈希

In [ ]:
import base64
import hashlib
import os

# 產生隨機的 code_verifier
code_verifier = base64.urlsafe_b64encode(os.urandom(32)).decode("utf-8").rstrip("=")

# 使用 SHA256 對 code_verifier 進行哈希
code_challenge = (
    base64.urlsafe_b64encode(hashlib.sha256(code_verifier.encode("utf-8")).digest())
    .decode("utf-8")
    .rstrip("=")
)

print(f"code_verifier: {code_verifier}")
print(f"code_challenge: {code_challenge}")

完整自動化腳本

In [7]:
import base64
import hashlib
import os
import json


def generate_code_verifier(length=64):
    """ 產生一個隨機的 code_verifier"""
    if length < 43 or length > 128:
        raise ValueError("code_verifier 長度必須在 43 至 128 位元組之間")
    code_verifier = (
        base64.urlsafe_b64encode(os.urandom(length)).decode("utf-8").rstrip("=")
    )
    return code_verifier


def generate_code_challenge(code_verifier):
    """使用 SHA-256 對 code_verifier 進行哈希，然後進行 Base64 URL 安全編碼"""
    code_challenge = (
        base64.urlsafe_b64encode(hashlib.sha256(code_verifier.encode("utf-8")).digest())
        .decode("utf-8")
        .rstrip("=")
    )
    return code_challenge


# 讀取 client_secret.json 文件
with open("client_secret.json", "r") as file:
    client_info = json.load(file)

# 獲取 client_id
client_id = client_info["installed"]["client_id"]

# 生成 code_verifier 和 code_challenge
code_verifier = generate_code_verifier()
code_challenge = generate_code_challenge(code_verifier)

# 生成 gcloud 命令
gcloud_command = (
    f'gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth'
    f"?response_type=code"
    f"&client_id={client_id}"
    f"&scope=https://www.googleapis.com/auth/cloud-platform+https://www.googleapis.com/auth/generative-language.tuning"
    f"&state=12345"
    f"&access_type=offline"
    f"&code_challenge={code_challenge}"
    f"&code_challenge_method=S256"
    f"&redirect_uri=http://localhost"
    f'&token_usage=remote"'
)

# 輸出 gcloud 命令
# print("Generated gcloud command:")
print(gcloud_command)

gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=826653416557-5f254qj34fu2kir6pl0bqin5jf8uifaf.apps.googleusercontent.com&scope=https://www.googleapis.com/auth/cloud-platform+https://www.googleapis.com/auth/generative-language.tuning&state=12345&access_type=offline&code_challenge=BSQQdKHKHZtd8thw9SVukAA7BUmODI5D6TrarDdCUgs&code_challenge_method=S256&redirect_uri=http://localhost&token_usage=remote"


### 使用腳本設置環境變數並查詢

In [8]:
import os
import json

# 讀取 client_secret.json 文件，並提取 project_id
with open('client_secret.json', 'r') as file:
    data = json.load(file)
    project_id = data['installed']['project_id']

# 獲取訪問令牌
access_token = !gcloud auth application-default print-access-token
access_token = '\n'.join(access_token)

# 設置環境變數
os.environ['access_token'] = access_token
os.environ['project_id'] = project_id
os.environ['base_url'] = "https://generativelanguage.googleapis.com"

# 驗證設置是否成功
print("Access Token:", os.environ['access_token'])
print("Project ID:", os.environ['project_id'])
print("Base URL:", os.environ['base_url'])

Access Token: ya29.a0AXooCgtvKsAQH3wuSO8YkqM3Mg8p9vFacOU2SDGuGWwtV5OPJyhdC1tXKMGMSEpyTvpqTuCkQnUX0B-PM3fNVvY1Gedru9MEpTjOHr-ooK5gi1aZoxkds3-NOtWUpp310NQH5hABBaEnaXWh2u7g0FHxHUC0GSp8nU9oaCgYKAY0SARASFQHGX2Mi28k0xvi7G1qbjF74WqmdPA0171
Project ID: myproject-20240622
Base URL: https://generativelanguage.googleapis.com


#### 僅查詢環境變數

In [5]:
# 驗證設置是否成功
print("Access Token:", os.environ['access_token'])
print("Project ID:", os.environ['project_id'])
print("Base URL:", os.environ['base_url'])

Access Token: ya29.a0AXooCgtg5b8ji5gm5Xm9el5gRpTCHthMP34ARze9-_2X-XNjvWdl2gq4QkBX5FJRdDjxbGs3kmsHiXdXaNcIwZJxB3sUmE8VBJ2mmqcBas_9C5aYVBdtTglfuMwujLwkRxZN9mHVOqFfToFDebB16IE8RLkwYGa3Rfe9aCgYKAQkSARASFQHGX2MiczW33061ZIF86F-IrDUZaQ0171
Project ID: myproject-20240622
Base URL: https://generativelanguage.googleapis.com


In [12]:
import json

# 解析 JSON 文件
first_page = json.load(open('tunemodel.json'))
os.environ['modelname'] = first_page['metadata']['tunedModel']

print(os.environ['modelname'])

tunedModels/number-generator-model-rmr2mn49025s


## 發送 REST API 請求

In [22]:
import requests
import json

access_token = !gcloud auth application-default print-access-token
access_token = '\n'.join(access_token)
# 替換自己的專案 ID
project = 'myproject-20240622'
base_url = "https://generativelanguage.googleapis.com"

#### 驗證認證設置

In [23]:
headers = {
    'Authorization': 'Bearer ' + access_token,
    'Content-Type': 'application/json',
    'x-goog-user-project': project
}

result = requests.get(
    url=f'{base_url}/v1beta/tunedModels',
    headers=headers,
)

print(result.json())

{'tunedModels': [{'name': 'tunedModels/number-generator-model-qw2t2pyfzld1', 'baseModel': 'models/gemini-1.0-pro-001', 'displayName': 'number generator model', 'state': 'ACTIVE', 'createTime': '2024-06-22T18:06:02.455154Z', 'updateTime': '2024-06-22T18:06:25.523912Z', 'tuningTask': {'startTime': '2024-06-22T18:06:02.997496412Z', 'completeTime': '2024-06-22T18:06:25.523912Z', 'snapshots': [{'step': 1, 'meanLoss': 11.499258, 'computeTime': '2024-06-22T18:06:04.430337734Z'}, {'step': 2, 'meanLoss': 13.731144, 'computeTime': '2024-06-22T18:06:04.890239450Z'}, {'step': 3, 'meanLoss': 8.956655, 'computeTime': '2024-06-22T18:06:05.448587324Z'}, {'step': 4, 'meanLoss': 12.012, 'computeTime': '2024-06-22T18:06:05.998061047Z'}, {'step': 5, 'meanLoss': 8.378381, 'computeTime': '2024-06-22T18:06:06.537827331Z'}, {'step': 6, 'meanLoss': 13.393811, 'computeTime': '2024-06-22T18:06:07.161799598Z'}, {'step': 7, 'meanLoss': 11.559576, 'computeTime': '2024-06-22T18:06:07.713890812Z'}, {'step': 8, 'epoch

#### 使用 `requests.post` 來傳入數據集

In [25]:
operation = requests.post(
    url=f"{base_url}/v1beta/tunedModels",
    headers=headers,
    json={
        "display_name": "number generator",
        "base_model": "models/gemini-1.0-pro-001",
        "tuning_task": {
            "hyperparameters": {
                "batch_size": 4,
                "learning_rate": 0.001,
                "epoch_count": 5,
            },
            "training_data": {
                "examples": {
                    "examples": [
                        {
                            "text_input": "1",
                            "output": "2",
                        },
                        {
                            "text_input": "3",
                            "output": "4",
                        },
                        {
                            "text_input": "-3",
                            "output": "-2",
                        },
                        {
                            "text_input": "twenty two",
                            "output": "twenty three",
                        },
                        {
                            "text_input": "two hundred",
                            "output": "two hundred one",
                        },
                        {
                            "text_input": "ninety nine",
                            "output": "one hundred",
                        },
                        {
                            "text_input": "8",
                            "output": "9",
                        },
                        {
                            "text_input": "-98",
                            "output": "-97",
                        },
                        {
                            "text_input": "1,000",
                            "output": "1,001",
                        },
                        {
                            "text_input": "10,100,000",
                            "output": "10,100,001",
                        },
                        {
                            "text_input": "thirteen",
                            "output": "fourteen",
                        },
                        {
                            "text_input": "eighty",
                            "output": "eighty one",
                        },
                        {
                            "text_input": "one",
                            "output": "two",
                        },
                        {
                            "text_input": "three",
                            "output": "four",
                        },
                        {
                            "text_input": "seven",
                            "output": "eight",
                        },
                    ]
                }
            },
        },
    },
)

if operation.status_code == 200:
    print('Success')

<Response [200]>

In [26]:
operation.json()

{'name': 'tunedModels/number-generator-g2dqvilkbvyl/operations/axpu7r5zmpo1',
 'metadata': {'@type': 'type.googleapis.com/google.ai.generativelanguage.v1beta.CreateTunedModelMetadata',
  'totalSteps': 19,
  'tunedModel': 'tunedModels/number-generator-g2dqvilkbvyl'}}

#### 使用調整後模型的名稱設定變數，以便用於其他呼叫

In [34]:
model_name=operation.json()["metadata"]["tunedModel"]
model_name

'tunedModels/number-generator-g2dqvilkbvyl'

#### 取得調整後模型狀態

In [35]:
tuned_model = requests.get(
    url = f'{base_url}/v1beta/{model_name}',
    headers=headers,
)
tuned_model.json()

{'name': 'tunedModels/number-generator-g2dqvilkbvyl',
 'baseModel': 'models/gemini-1.0-pro-001',
 'displayName': 'number generator',
 'state': 'ACTIVE',
 'createTime': '2024-06-22T19:16:26.308796Z',
 'updateTime': '2024-06-22T19:17:17.806207Z',
 'tuningTask': {'startTime': '2024-06-22T19:16:26.737766278Z',
  'completeTime': '2024-06-22T19:17:17.806207Z',
  'snapshots': [{'step': 1,
    'meanLoss': 12.668232,
    'computeTime': '2024-06-22T19:16:28.697343631Z'},
   {'step': 2,
    'meanLoss': 10.72086,
    'computeTime': '2024-06-22T19:16:30.202175170Z'},
   {'step': 3,
    'meanLoss': 11.8869,
    'computeTime': '2024-06-22T19:16:32.484395627Z'},
   {'step': 4,
    'epoch': 1,
    'meanLoss': 10.7399435,
    'computeTime': '2024-06-22T19:16:34.105591398Z'},
   {'step': 5,
    'epoch': 1,
    'meanLoss': 11.980744,
    'computeTime': '2024-06-22T19:16:36.835052700Z'},
   {'step': 6,
    'epoch': 1,
    'meanLoss': 8.332833,
    'computeTime': '2024-06-22T19:16:38.570319007Z'},
   {'step

#### 每 5 秒檢查一次狀態欄位，直到狀態不再處於 CREATING 狀態為止

In [30]:
import time
import pprint

op_json = operation.json()
response = op_json.get("response")
error = op_json.get("error")

while response is None and error is None:
    time.sleep(5)

    operation = requests.get(
        url=f'{base_url}/v1/{op_json["name"]}',
        headers=headers,
    )

    op_json = operation.json()
    response = op_json.get("response")
    error = op_json.get("error")

    percent = op_json["metadata"].get("completedPercent")
    if percent is not None:
        print(f"{percent:.2f}% - {op_json['metadata']['snapshots'][-1]}")
        print()

if error is not None:
    raise Exception(error)

100.00% - {'step': 19, 'epoch': 5, 'meanLoss': 0.9600375, 'computeTime': '2024-06-22T19:17:17.506261066Z'}



## 執行推論

In [32]:
import time

m = requests.post(
    url=f"{base_url}/v1beta/{name}:generateContent",
    headers=headers,
    json={"contents": [{"parts": [{"text": "六"}]}]},
)
import pprint

pprint.pprint(m.json())

{'candidates': [{'content': {'parts': [{'text': '七'}], 'role': 'model'},
                 'finishReason': 'STOP',
                 'index': 0,
                 'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
                                    'probability': 'NEGLIGIBLE'},
                                   {'category': 'HARM_CATEGORY_HATE_SPEECH',
                                    'probability': 'NEGLIGIBLE'},
                                   {'category': 'HARM_CATEGORY_HARASSMENT',
                                    'probability': 'NEGLIGIBLE'},
                                   {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
                                    'probability': 'NEGLIGIBLE'}]}],
 'usageMetadata': {'candidatesTokenCount': 1,
                   'promptTokenCount': 1,
                   'totalTokenCount': 2}}
